Imports

In [11]:
import osmnx as ox
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [54]:
query = '''
    /*
    This has been generated by the overpass-turbo wizard.
    The original search was:
    “tourism=* in Manhattan”
    */
    [out:csv( ::"id",::"type",::"lat",::"lon",name, tourism, wikipedia)][timeout:25];
    // fetch area “Manhattan” to search in
    area(id:3608398124)->.searchArea;
    // gather results
    nwr["tourism"](area.searchArea);
    // print results
    out geom;
'''

response = requests.post(url='https://overpass-api.de/api/interpreter',data=query)
print(response.status_code)
print(response.text)
with open("data/OSM_Manhattan_POI.csv",'w',encoding='utf8') as file:
    file.write(response.text)

200
@id	@type	@lat	@lon	name	tourism	wikipedia
158801311	node	40.7251022	-73.9795833	Alphabet City	yes	
158809984	node	40.7228800	-73.9987505	SoHo	attraction	"en:SoHo, Manhattan"
158862484	node	40.7486231	-73.9713889	Tudor City	yes	
320948228	node	40.7057505	-74.0029056	South Street Seaport	attraction	en:South Street Seaport
357618558	node	40.7322674	-73.9972548	"New York Studio School of Drawing, Painting and Sculpture"	yes	"en:New York Studio School of Drawing, Painting and Sculpture"
357620718	node	40.7714555	-73.9679875	Richard Morris Hunt	artwork	en:Richard Morris Hunt Memorial
357621823	node	40.7941111	-73.9764799	Joan of Arc Memorial	artwork	
368042980	node	40.7214798	-73.9971028	Storefront for Art and Architecture	gallery	en:Storefront for Art and Architecture
368043598	node	40.7338007	-74.0021484	Stonewall Inn	attraction	en:Stonewall Inn
368043670	node	40.7470923	-74.0064354	Paula Cooper Gallery	gallery	
368052081	node	40.7033870	-74.0113540	Fraunces Tavern Museum	museum	
3680

In [43]:
response = requests.get('https://overpass-api.de/api/interpreter?data=%2F*%0AThis%20has%20been%20generated%20by%20the%20overpass-turbo%20wizard.%0AThe%20original%20search%20was%3A%0A%E2%80%9Ctourism%3D*%20in%20Manhattan%E2%80%9D%0A*%2F%0A%5Bout%3Ajson%5D%5Btimeout%3A25%5D%3B%0A%2F%2F%20fetch%20area%20%E2%80%9CManhattan%E2%80%9D%20to%20search%20in%0Aarea%28id%3A3608398124%29-%3E.searchArea%3B%0A%2F%2F%20gather%20results%0Anwr%5B%22tourism%22%5D%28area.searchArea%29%3B%0A%2F%2F%20print%20results%0Aout%20geom%3B')
print(response.status_code)
data = response.json()
with open("data/raw/OSM_Manhattan_Tourism",'w',encoding='utf8') as file:
    file.write(str(data))

200


In [55]:
query = '''
/*
This has been generated by the overpass-turbo wizard.
The original search was:
“(highway=primary or highway=secondary) and type:way in manhattan”
*/
[out:csv( ::"id",::"type",::"lat",::"lon",name, highway, lanes,maxspeed,oneway)][timeout:25];
// fetch area “manhattan” to search in
area(id:3608398124)->.searchArea;
// gather results
(
//this line MODIFIED
way(area.searchArea)["highway"~"primary|secondary|service|residential|tertiary|unclassified|living_street"];
);
// print results
out geom;
'''

response = requests.post(url='https://overpass-api.de/api/interpreter',data=query)
print(response.status_code)
print(response.text)
with open("data/OSM_Manhattan_Streets.csv",'w',encoding='utf8') as file:
    file.write(response.text)

200
@id	@type	@lat	@lon	name	highway	lanes	maxspeed	oneway
5668966	way	40.7983509	-73.9607645	West 106th Street	secondary			
5668968	way	40.7841149	-73.9785054	West 80th Street	residential	2		yes
5668973	way	40.8598859	-73.9283292	Hillside Avenue	tertiary			yes
5668977	way	40.7868339	-73.9771099	West 84th Street	residential			yes
5668983	way	40.7259372	-73.9757589	Szold Place	residential		20 mph	yes
5668986	way	40.8134030	-73.9587581	La Salle Street	residential			
5668989	way	40.7334520	-73.9973157	West 9th Street	residential	1	25 mph	yes
5668993	way	40.7349901	-73.9993119	Patchin Place	service		25 mph	
5668997	way	40.7257535	-73.9939867	Jones Alley	service			
5668999	way	40.7041428	-74.0086478	Old Slip	residential	1	20 mph	yes
5669002	way	40.7702685	-73.9833606	West 61st Street	residential	2		yes
5669004	way	40.7713099	-73.9859587	West 61st Street	residential	2	25 mph	no
5669008	way	40.7895114	-73.9732797	West 89th Street	residential			yes
5669013	way	40.7112390	-74.0149586	Liberty St

In [ ]:
response = requests.get('https://overpass-api.de/api/interpreter?data=%5Bout%3Ajson%5D%5Btimeout%3A25%5D%3B%0Aarea%28id%3A3608398124%29-%3E.searchArea%3B%0A%28%0A%0Away%28area.searchArea%29%5Bhighway~%22%5Eprimary%7Csecondary%7Cfootway%7Cservice%7Cresidential%7Ctertiary%7Cunclassified%7Cpath%7Ctrack%7Cliving_street%22%5D%5Bname%5D%3B%0A%29%3B%0A%0Aout%20geom%3B')
print(response.status_code)
data = response.json()
with open("data/raw/OSM_Manhattan_Streets",'w',encoding='utf8') as file:
    file.write(str(data))

200


In [13]:
def process_osm_data(osm_data):
    street_data = []
    
    for element in osm_data['elements']:
        if 'tags' in element and 'maxspeed' in element['tags']:
            street = {
                'osm_id': element['id'],
                'highway': element['tags'].get('highway', ''),
                'maxspeed': element['tags'].get('maxspeed', 'unknown')
            }
            street_data.append(street)
    
    df = pd.DataFrame(street_data)
    return df

In [14]:
def visualize_streets(city_name):
    G = ox.graph_from_place(city_name, network_type='all')

    ox.plot_graph(G, figsize=(10, 10))
    plt.show()

In [17]:
city_name = "New York, New York, USA"
    
osm_data = get_osm_data(city_name)
print(osm_data)
df = process_osm_data(osm_data)
    
print(df.head())
visualize_streets(city_name)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)